### Setting the working directory

Before loading the data, let's begin by setting the right working directory. In order to change the working directory, we use the `os` library:

In [ ]:
import os
os.getcwd()

'/content'

Use the following line to change the working directory to the path where our python files and data files are saved in:

(The **new_path** is where I stored the data in my Google Drive. To run the code successfully, please create the necessary folder in your Google Drive and upload related data there)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
new_path = '/content/drive/My Drive/IBM Mangrove Project'
os.chdir(new_path)

In [ ]:
# Verify the current working directory
print("Current Working Directory:", os.getcwd())

In [ ]:
India_And_Indonesia_Data = pd.read_excel('India and Indonesia dataADB.xlsx', 'India and Indonesia dataADB')
Mangrove_by_country = pd.read_excel('gmw_v3_country_statistics_ha.xlsx', '(%) Country Change From 1996')

### Importing Libraries

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from itertools import product
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
import matplotlib.pyplot as plt

### **India**: Importing, Preparing, and Transforming the Data

Then, let's import and prepare our data for the regressions:

In [ ]:
# Install necessary libraries (if not already available in Colab)
!pip install openpyxl  # If using Excel files that require openpyxl

In [ ]:
India_And_Indonesia_Data = pd.read_excel('/content/India and Indonesia dataADB.xlsx')
Mangrove_by_country = pd.read_excel('/content/gmw_v3_country_statistics_ha.xlsx')

In [ ]:
India_And_Indonesia_Data.head()

,Economy,Indicator,Unit of Measure,2000,2001,2002,2003,2004,2005,2006,...,2019,2020,2021,2022,Definition,Data Coverage,Calendar Year,Base Year,Source,Footnotes
0,Indonesia,GDP at current prices,Indonesian Rupiah,1389769850000000,1646322100000000,1821833360000000,2013674600000000,2295826200000000,2774281100000000,3339216800000000,...,15832657200000000,15438017500000000,16970789200000000,...,Unduplicated market value of the total product...,From 2000 to 2022,Calendar Year,NaN,BPS Statistics Indonesia,NaN
1,Indonesia,Agriculture (% of GDP),percent of GDP,15.60197,15.29026,15.45645,15.18535,14.33578,13.12662,12.9738,...,13.258161,14.221554,13.840219,...,Value-added of the agricultural sector as perc...,From 2000 to 2022,Calendar Year,NaN,BPS Statistics Indonesia,NaN
2,Indonesia,Industry (% of GDP),percent of GDP,45.9254,46.45484,44.46292,43.74957,44.62762,46.54106,46.94356,...,40.623372,39.700684,41.538108,...,Value-added of the industry sector as percent ...,From 2000 to 2022,Calendar Year,NaN,BPS Statistics Indonesia,NaN
3,Indonesia,Per capita GDP,Indonesian Rupiah,6737801.11,7890613.68,8616358.48,9398657.56,10576110.92,12618857.69,14991080.91,...,59060096.61,56938722.67,62236558.33535,...,"GDP at current prices, divided by the midyear ...",From 2000 to 2022,Calendar Year,NaN,2000–2020: BPS Statistics Indonesia. 2021: Asi...,NaN
4,Indonesia,"Road Indicators Network, Total (km)",kilometer,348083,352762,357026,357959,372928,391008,406569,...,544474,548366,...,...,This includes both paved and unpaved roads. Pa...,From 2000 to 2022,NaN,NaN,Asian Development Bank,NaN


In [ ]:
Mangrove_by_country.head()

,C_ID,Name,1996,2007,2008,2009,2010,2015,2016,2017,2018,2019,2020
0,ABW,Aruba,55.052209,48.193675,45.427293,44.921545,45.199606,46.310394,46.310394,46.310394,44.186668,44.186668,45.938704
1,AGO,Angola,29325.123955,28980.640435,28835.877511,28868.925084,28844.644401,28792.705166,28554.688658,28567.111454,28357.653566,28438.622013,28356.673109
2,AIA,Anguilla,4.278923,3.338858,3.204173,3.342982,3.981207,4.706479,4.612542,4.427438,3.884537,3.610993,3.700790
3,ARE,United Arab Emirates,7582.869698,7953.942921,8406.867488,8217.897942,7706.295157,7274.337113,7194.459970,7325.904706,7422.617370,7455.832152,7444.860192
4,ASM,American Samoa,32.599267,32.461965,32.324667,32.141598,32.187365,32.416202,32.416202,32.416202,32.416202,32.278906,32.050071


In [ ]:
India_data_1 = India_And_Indonesia_Data[India_And_Indonesia_Data['Economy'] == 'India']
print(India_data_1.shape)
India_data_1.head()

(53, 32)


,Economy,Indicator,Unit of Measure,2000,2001,2002,2003,2004,2005,2006,...,2019,2020,2021,2022,Definition,Data Coverage,Calendar Year,Base Year,Source,Footnotes
49,India,"Road Indicators Network, Total (km)",kilometer,3325765,3373520,3426600,3528654,3621507,3809156,3880651,...,6386297,...,...,...,This includes both paved and unpaved roads. Pa...,From 2000 to 2022,NaN,NaN,Asian Development Bank,NaN
50,India,"Rail Lines, Total Route (km)",kilometer,63028,63140,63122,63221,63465,63332,63327,...,67956,...,...,...,Rail lines are the length of railway route ava...,From 2000 to 2022,NaN,NaN,Asian Development Bank,NaN
51,India,Total expenditure,Indian Rupee,...,...,...,...,...,...,...,...,50038650000000,63040247000000,...,...,NaN,From 2000 to 2022,Beginning 0401,NaN,National Statistical Office,NaN
52,India,General public services,Indian Rupee,...,...,...,...,...,...,...,...,6675360000000,9685615000000,...,...,NaN,From 2000 to 2022,Beginning 0401,NaN,National Statistical Office,NaN
53,India,Defense,Indian Rupee,...,...,...,...,...,...,...,...,5015720000000,5446966000000,...,...,NaN,From 2000 to 2022,Beginning 0401,NaN,National Statistical Office,NaN


In [ ]:
India_data_2 = Mangrove_by_country[Mangrove_by_country['Name'] == 'India']
print(India_data_2.shape)
India_data_2.head()

(1, 13)


,C_ID,Name,1996,2007,2008,2009,2010,2015,2016,2017,2018,2019,2020
53,IND,India,411118.55768,406149.587179,403223.125799,403398.892877,402496.269092,406178.10467,408113.520715,408676.493082,409495.036444,408123.375173,403784.617585


In [ ]:
# Set the first column as the index for transposition if it's not already set
India_data_1_copy_1 = India_data_1.set_index('Indicator')

# Transpose the DataFrame
India_data_1_transposed = India_data_1_copy_1.T

# Reset the index to turn the 'Year' index into a column
India_data_1_transposed = India_data_1_transposed.reset_index().rename(columns={'index': 'Year'})

print(India_data_1_transposed.shape)
India_data_1_transposed.head()

(31, 54)


Indicator,Year,"Road Indicators Network, Total (km)","Rail Lines, Total Route (km)",Total expenditure,General public services,Defense,Public order and safety,Economic affairs,Environmental protection,Housing and community amenities,...,Proportion of Population Covered by 2G Mobile Networks (%),Proportion of Population Covered by 3G Mobile Networks (%),Proportion of Population Covered by LTE Mobile Networks (%),"Annual Mean Levels (μg/m³) of Fine Particulate Matter (e.g., PM2.5 and PM10) in Cities (population weighted), Urban","Annual Mean Levels (μg/m³) of Fine Particulate Matter (e.g., PM2.5 and PM10) in Cities (population weighted), Total",Direct Economic Loss Attributed to Disasters ($ million),"Proportion of Urban Population Living in Slums, Informal Settlements, or Inadequate Housing (%)",Coverage of Protected Areas in Relation to Marine Areas (Exclusive Economic Zones) (%),Protected Marine Areas (Exclusive Economic Zones) (km²),Number of Persons Affected by Disaster
0,Economy,India,India,India,India,India,India,India,India,India,...,India,India,India,India,India,India,India,India,India,India
1,Unit of Measure,kilometer,kilometer,Indian Rupee,Indian Rupee,Indian Rupee,Indian Rupee,Indian Rupee,Indian Rupee,Indian Rupee,...,percent,percent,percent,micrograms per cubic meter,micrograms per cubic meter,US Dollar,percent,percent,square kilometer,persons
2,2000,3325765,63028,...,...,...,...,...,...,...,...,21.1,...,...,...,...,...,55.29477,...,...,...
3,2001,3373520,63140,...,...,...,...,...,...,...,...,21.1,...,...,...,...,...,...,...,...,...
4,2002,3426600,63122,...,...,...,...,...,...,...,...,...,...,...,...,...,...,54.59656,...,...,...


The transposed data looks messy in format, as it has columns and rows that we don't need in future analysis. Let's reorganize them so that the data can be merged more easily later:

In [ ]:
# Create a copy so that we don't mess up the original data
India_data_1_transposed_copy = India_data_1_transposed.copy()

# Convert 'Year' to string type to ensure the str methods can be applied
India_data_1_transposed_copy['Year'] = India_data_1_transposed_copy['Year'].astype(str)

# Filter out rows where 'Year' does not match the four-digit pattern and is not NaN
India_data_1_cleaned = India_data_1_transposed_copy[
    India_data_1_transposed_copy['Year'].str.match(r'^\d{4}$', na=False)
]

# Now convert the 'Year' column to integer since we have filtered out non-year values
India_data_1_cleaned['Year'] = India_data_1_cleaned['Year'].astype(int)

# Set 'Year' as the index
India_data_1_cleaned.set_index('Year', inplace=True)

print(India_data_1_cleaned.shape)
India_data_1_cleaned.head()

(23, 53)


<ipython-input-10-b6157e00d455>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  India_data_1_cleaned['Year'] = India_data_1_cleaned['Year'].astype(int)


Indicator,"Road Indicators Network, Total (km)","Rail Lines, Total Route (km)",Total expenditure,General public services,Defense,Public order and safety,Economic affairs,Environmental protection,Housing and community amenities,Expenditure—Health,...,Proportion of Population Covered by 2G Mobile Networks (%),Proportion of Population Covered by 3G Mobile Networks (%),Proportion of Population Covered by LTE Mobile Networks (%),"Annual Mean Levels (μg/m³) of Fine Particulate Matter (e.g., PM2.5 and PM10) in Cities (population weighted), Urban","Annual Mean Levels (μg/m³) of Fine Particulate Matter (e.g., PM2.5 and PM10) in Cities (population weighted), Total",Direct Economic Loss Attributed to Disasters ($ million),"Proportion of Urban Population Living in Slums, Informal Settlements, or Inadequate Housing (%)",Coverage of Protected Areas in Relation to Marine Areas (Exclusive Economic Zones) (%),Protected Marine Areas (Exclusive Economic Zones) (km²),Number of Persons Affected by Disaster
Year,,,,,,,,,,,,,,,,,,,,,
2000,3325765,63028,...,...,...,...,...,...,...,...,...,21.1,...,...,...,...,...,55.29477,...,...,...
2001,3373520,63140,...,...,...,...,...,...,...,...,...,21.1,...,...,...,...,...,...,...,...,...
2002,3426600,63122,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,54.59656,...,...,...
2003,3528654,63221,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004,3621507,63465,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,53.89834,...,...,...


Now let's transform data 2:

In [ ]:
print(India_data_2.shape)
India_data_2.head()

(1, 13)


,C_ID,Name,1996,2007,2008,2009,2010,2015,2016,2017,2018,2019,2020
53,IND,India,411118.55768,406149.587179,403223.125799,403398.892877,402496.269092,406178.10467,408113.520715,408676.493082,409495.036444,408123.375173,403784.617585


In [ ]:
# Drop the 'C_ID' and 'Name' columns
India_data_2_dropped = India_data_2.drop(['C_ID', 'Name'], axis=1)

# Transpose the remaining data
India_data_2_transposed = India_data_2_dropped.T

# Reset the index to make 'Year' a column
India_data_2_transposed = India_data_2_transposed.reset_index()

# Rename the columns appropriately
India_data_2_transposed.columns = ['Year', 'Mangrove Change From 1996']

# Ensure 'Year' is of integer type
India_data_2_transposed['Year'] = India_data_2_transposed['Year'].astype(int)

# Set 'Year' as the index
India_data_2_transposed.set_index('Year', inplace=True)

print(India_data_2_transposed.shape)
India_data_2_transposed.head()

(11, 1)


,Mangrove Change From 1996
Year,
1996,411118.557680
2007,406149.587179
2008,403223.125799
2009,403398.892877
2010,402496.269092


### **India**: Inspecting the Data

In [ ]:
# Join the two DataFrames on their indices
merged_India_data = India_data_1_cleaned.join(India_data_2_transposed, how='left')

# Ensure the 'Mangrove Change From 1996' is the first column
cols = ['Mangrove Change From 1996'] + [col for col in merged_India_data.columns if col != 'Mangrove Change From 1996']
merged_India_data = merged_India_data[cols]

print(merged_India_data.shape)
merged_India_data.head()

(23, 54)


,Mangrove Change From 1996,"Road Indicators Network, Total (km)","Rail Lines, Total Route (km)",Total expenditure,General public services,Defense,Public order and safety,Economic affairs,Environmental protection,Housing and community amenities,...,Proportion of Population Covered by 2G Mobile Networks (%),Proportion of Population Covered by 3G Mobile Networks (%),Proportion of Population Covered by LTE Mobile Networks (%),"Annual Mean Levels (μg/m³) of Fine Particulate Matter (e.g., PM2.5 and PM10) in Cities (population weighted), Urban","Annual Mean Levels (μg/m³) of Fine Particulate Matter (e.g., PM2.5 and PM10) in Cities (population weighted), Total",Direct Economic Loss Attributed to Disasters ($ million),"Proportion of Urban Population Living in Slums, Informal Settlements, or Inadequate Housing (%)",Coverage of Protected Areas in Relation to Marine Areas (Exclusive Economic Zones) (%),Protected Marine Areas (Exclusive Economic Zones) (km²),Number of Persons Affected by Disaster
Year,,,,,,,,,,,,,,,,,,,,,
2000,NaN,3325765,63028,...,...,...,...,...,...,...,...,21.1,...,...,...,...,...,55.29477,...,...,...
2001,NaN,3373520,63140,...,...,...,...,...,...,...,...,21.1,...,...,...,...,...,...,...,...,...
2002,NaN,3426600,63122,...,...,...,...,...,...,...,...,...,...,...,...,...,...,54.59656,...,...,...
2003,NaN,3528654,63221,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004,NaN,3621507,63465,...,...,...,...,...,...,...,...,...,...,...,...,...,...,53.89834,...,...,...


In [ ]:
#merged_India_data.to_csv('/content/drive/My Drive/IBM Mangrove Project/transposed India Data.csv', index=True)

###Some Thinking:

By inspecting the merged data from above, we believe that there exist the following challenges:

1. **The sample size is too small.** <br>
Since we treat each year as an observation and we only have 10 years of mangrove loss data, we ended up with only 10 observations that we can use to run OLS and construct predictive models, and even further separating it into training and testing set. We could face very high risk of overfitting. <br>
<br>
2. **Too many features.** <br>
We have nearly 50 features from the data above, which could cause potential overfitting or multi-colinearity issues when running regressions. Also, it might lower our efficiency during feature selection.<br>
<br>
3. **Too many missing (NaN) values.** <br>
It poses challenges for us to handle these missing values with such a small sample size.
<br>

Therefore, a potential solution could be **incorporating data from more countries across time**, which would bring us a larger sample size and may resolve some of the challenges we face. For now, let's forego the idea of building a tailored predictive model for each country, but focus on building **a master predictive model for all countries**. Let's try adding the India data as our very first next step.

### Preparing for regressions

In [ ]:
merged_India_cleaned = merged_India_data.dropna(subset=['Mangrove Change From 1996']).reset_index(drop=True).iloc[:, :-1]
print(merged_India_cleaned.shape)
merged_India_cleaned

(10, 53)


,Mangrove Change From 1996,"Road Indicators Network, Total (km)","Rail Lines, Total Route (km)",Total expenditure,General public services,Defense,Public order and safety,Economic affairs,Environmental protection,Housing and community amenities,...,"Level of Water Stress, Freshwater Withdrawal as a Proportion of Available Freshwater Resources (%)",Proportion of Population Covered by 2G Mobile Networks (%),Proportion of Population Covered by 3G Mobile Networks (%),Proportion of Population Covered by LTE Mobile Networks (%),"Annual Mean Levels (μg/m³) of Fine Particulate Matter (e.g., PM2.5 and PM10) in Cities (population weighted), Urban","Annual Mean Levels (μg/m³) of Fine Particulate Matter (e.g., PM2.5 and PM10) in Cities (population weighted), Total",Direct Economic Loss Attributed to Disasters ($ million),"Proportion of Urban Population Living in Slums, Informal Settlements, or Inadequate Housing (%)",Coverage of Protected Areas in Relation to Marine Areas (Exclusive Economic Zones) (%),Protected Marine Areas (Exclusive Economic Zones) (km²)
0,406149.587179,4016401,63273,...,...,...,...,...,...,...,...,65.34913,70,0,...,...,...,...,...,...,...
1,403223.125799,4109592,64015,...,...,...,...,...,...,...,...,65.73012,...,0,...,...,...,...,52.50192,...,...
2,403398.892877,4471510,63974,...,...,...,...,...,...,...,...,66.11111,83,0.08,...,...,...,...,...,...,...
3,402496.269092,4582439,64460,...,...,...,...,...,...,...,...,66.49209,...,...,...,69.151169,64.830207,...,51.8037,...,...
4,406178.104670,5472144,66252,33907340000000,4640880000000,3264430000000,1951020000000,11076820000000,17910000000,2545880000000,...,66.49209,95,74,4,69.162473,64.917532,0,...,...,...
5,408113.520715,5603293,66918,37166550000000,4773440000000,3787090000000,2233240000000,11681570000000,15450000000,3177550000000,...,66.49209,96,79.67,73.5,69.640429,65.409503,0,49.70906,...,...
6,408676.493082,5909561,66935,41706567100000,4857245500000,4219102200000,2571102800000,13309948800000,23898700000,4053687200000,...,66.49209,97,88,88,70.270798,66.032088,0,...,...,...
7,409495.036444,6215797,67415,45730447000000,5755813000000,4548675000000,2953301000000,14368059000000,41165000000,3880511000000,...,66.49209,97,94,94,72.234019,67.867429,...,49.01085,...,...
8,408123.375173,6386297,67956,50038650000000,6675360000000,5015720000000,3221267000000,15643604000000,35102000000,3659731000000,...,66.49209,99.06,98.15,97.92,62.363661,58.511269,0,...,...,...
9,403784.617585,...,...,63040247000000,9685615000000,5446966000000,3495983000000,21754391000000,41664000000,4802885000000,...,...,99.12,98.56,98.45,...,...,...,49.01085,...,...


In [ ]:
def remove_sparse_columns(data_frame, threshold_proportion):
    """
    Removes columns from the DataFrame where the proportion of non-NaN/non-placeholder values
    is less than the specified threshold. Additionally, it removes any columns labeled NaN if they exist.

    Parameters:
    - data_frame (pd.DataFrame): The input DataFrame.
    - threshold_proportion (float): The threshold proportion of non-NaN/non-placeholder values required to keep the column.

    Returns:
    - pd.DataFrame: A DataFrame with sparse columns and any NaN-labeled columns removed.
    """
    # Convert placeholders and non-numeric values to NaN
    data_frame = data_frame.replace('...', np.nan)  # Replace any '...' placeholders with NaN
    data_frame = data_frame.apply(pd.to_numeric, errors='coerce')  # Coerce non-numeric values to NaN

    # Check for and remove columns labeled NaN
    nan_columns = data_frame.columns[data_frame.columns.isnull()]
    if not nan_columns.empty:
        data_frame = data_frame.drop(columns=nan_columns)

    # Calculate the minimum count of non-NaN values required
    min_count = int(len(data_frame) * threshold_proportion)

    # Create a boolean mask where True indicates the columns to keep
    cols_to_keep = data_frame.apply(lambda col: col.count() >= min_count)

    # Select the columns to keep based on the boolean mask
    cleaned_df = data_frame.loc[:, cols_to_keep]

    return cleaned_df

In [ ]:
merged_India_cleaned_copy = remove_sparse_columns(merged_India_cleaned, 0.8)
print(merged_India_cleaned_copy.shape)
merged_India_cleaned_copy

(10, 28)


,Mangrove Change From 1996,"Road Indicators Network, Total (km)","Rail Lines, Total Route (km)",International Tourist Arrivals ('000),International Tourism Receipts ($ million),GDP at current prices,Agriculture (% of GDP),Industry (% of GDP),Per capita GDP,Deforestation Rate (average % change),...,"Carbon Dioxide Emissions, Per Unit of Manufacturing Value-Added (kg of CO₂ equivalent per constant 2015 $)","Carbon Dioxide Emissions, Per Unit of GDP (PPP) (kg of CO₂ equivalent per constant 2017 $)",Number of Deaths Due to Disaster,"Proportion of Population Using Safely Managed Drinking Water, Rural (%)","Proportion of Population Using Safely Managed Sanitation, Total (%)","Proportion of Population Using Safely Managed Sanitation, Urban (%)","Proportion of Population Using Safely Managed Sanitation, Rural (%)","Level of Water Stress, Freshwater Withdrawal as a Proportion of Available Freshwater Resources (%)",Proportion of Population Covered by 2G Mobile Networks (%),Proportion of Population Covered by 3G Mobile Networks (%)
0,406149.587179,4016401.0,63273.0,5082000,10730000000,48986620590000,18.273885,33.714518,43046.23953,-0.28,...,1.510,0.292,8340,39,19,27,16,65.34913,70.00,0.00
1,403223.125799,4109592.0,64015.0,5283000,11832000000,55141523850000,17.898843,33.191183,47782.94961,-0.28,...,1.505,0.302,7341,40,21,28,19,65.73012,NaN,0.00
2,403398.892877,4471510.0,63974.0,5168000,11136000000,63664065420000,17.841424,33.160573,54413.73113,-0.28,...,1.564,0.308,6148,42,23,28,21,66.11111,83.00,0.08
3,402496.269092,4582439.0,64460.0,5776000,14490000000,76344721040000,18.350431,33.114153,64371.60290,-0.27,...,1.628,0.301,7489,43,25,29,24,66.49209,NaN,NaN
4,406178.104670,5472144.0,66252.0,13284000,21013000000,137718740000000,17.714686,29.951484,107341.18380,-0.38,...,1.487,0.284,7313,51,36,33,37,66.49209,95.00,74.00
5,408113.520715,5603293.0,66918.0,14570000,22427000000,153916690000000,18.035273,29.337983,118488.59900,-0.38,...,1.435,0.267,6942,52,38,34,40,66.49209,96.00,79.67
6,408676.493082,5909561.0,66935.0,15543000,27365000000,170900423600000,18.250271,29.207802,130061.20520,-0.37,...,1.444,0.265,7317,54,40,35,42,66.49209,97.00,88.00
7,409495.036444,6215797.0,67415.0,17423000,28568000000,188996684400000,17.641353,29.064108,142424.02750,-0.37,...,1.349,0.262,6859,56,42,36,45,66.49209,97.00,94.00
8,408123.375173,6386297.0,67956.0,17914000,30720000000,200748557900000,18.296615,26.900975,149700.63980,-0.37,...,1.416,0.252,7505,57,44,37,48,66.49209,99.06,98.15
9,403784.617585,NaN,NaN,6330000,13036000000,198009138200000,19.988549,26.899028,146087.24710,NaN,...,NaN,NaN,6347,56,46,37,51,NaN,99.12,98.56


### Inspecting the Pre-regression Data

In [ ]:
# Create a copy of the DataFrame to preserve the original data
df_cleaned_copy = merged_India_cleaned_copy.copy()

# Initialize the imputer to replace NaN values with the mean of each column
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# Perform the imputation on the DataFrame copy
prereg_df_imputed = pd.DataFrame(imputer.fit_transform(df_cleaned_copy), columns=df_cleaned_copy.columns)

# Display the first few rows to verify the imputation
print(prereg_df_imputed.shape)
prereg_df_imputed

(10, 28)


,Mangrove Change From 1996,"Road Indicators Network, Total (km)","Rail Lines, Total Route (km)",International Tourist Arrivals ('000),International Tourism Receipts ($ million),GDP at current prices,Agriculture (% of GDP),Industry (% of GDP),Per capita GDP,Deforestation Rate (average % change),...,"Carbon Dioxide Emissions, Per Unit of Manufacturing Value-Added (kg of CO₂ equivalent per constant 2015 $)","Carbon Dioxide Emissions, Per Unit of GDP (PPP) (kg of CO₂ equivalent per constant 2017 $)",Number of Deaths Due to Disaster,"Proportion of Population Using Safely Managed Drinking Water, Rural (%)","Proportion of Population Using Safely Managed Sanitation, Total (%)","Proportion of Population Using Safely Managed Sanitation, Urban (%)","Proportion of Population Using Safely Managed Sanitation, Rural (%)","Level of Water Stress, Freshwater Withdrawal as a Proportion of Available Freshwater Resources (%)",Proportion of Population Covered by 2G Mobile Networks (%),Proportion of Population Covered by 3G Mobile Networks (%)
0,406149.587179,4.016401e+06,63273.000000,5082000.0,1.073000e+10,4.898662e+13,18.273885,33.714518,43046.23953,-0.280000,...,1.510,0.292000,8340.0,39.0,19.0,27.0,16.0,65.34913,70.0000,0.000000
1,403223.125799,4.109592e+06,64015.000000,5283000.0,1.183200e+10,5.514152e+13,17.898843,33.191183,47782.94961,-0.280000,...,1.505,0.302000,7341.0,40.0,21.0,28.0,19.0,65.73012,92.0225,0.000000
2,403398.892877,4.471510e+06,63974.000000,5168000.0,1.113600e+10,6.366407e+13,17.841424,33.160573,54413.73113,-0.280000,...,1.564,0.308000,6148.0,42.0,23.0,28.0,21.0,66.11111,83.0000,0.080000
3,402496.269092,4.582439e+06,64460.000000,5776000.0,1.449000e+10,7.634472e+13,18.350431,33.114153,64371.60290,-0.270000,...,1.628,0.301000,7489.0,43.0,25.0,29.0,24.0,66.49209,92.0225,59.162222
4,406178.104670,5.472144e+06,66252.000000,13284000.0,2.101300e+10,1.377187e+14,17.714686,29.951484,107341.18380,-0.380000,...,1.487,0.284000,7313.0,51.0,36.0,33.0,37.0,66.49209,95.0000,74.000000
5,408113.520715,5.603293e+06,66918.000000,14570000.0,2.242700e+10,1.539167e+14,18.035273,29.337983,118488.59900,-0.380000,...,1.435,0.267000,6942.0,52.0,38.0,34.0,40.0,66.49209,96.0000,79.670000
6,408676.493082,5.909561e+06,66935.000000,15543000.0,2.736500e+10,1.709004e+14,18.250271,29.207802,130061.20520,-0.370000,...,1.444,0.265000,7317.0,54.0,40.0,35.0,42.0,66.49209,97.0000,88.000000
7,409495.036444,6.215797e+06,67415.000000,17423000.0,2.856800e+10,1.889967e+14,17.641353,29.064108,142424.02750,-0.370000,...,1.349,0.262000,6859.0,56.0,42.0,36.0,45.0,66.49209,97.0000,94.000000
8,408123.375173,6.386297e+06,67956.000000,17914000.0,3.072000e+10,2.007486e+14,18.296615,26.900975,149700.63980,-0.370000,...,1.416,0.252000,7505.0,57.0,44.0,37.0,48.0,66.49209,99.0600,98.150000
9,403784.617585,5.196337e+06,65688.666667,6330000.0,1.303600e+10,1.980091e+14,19.988549,26.899028,146087.24710,-0.331111,...,1.482,0.281444,6347.0,56.0,46.0,37.0,51.0,66.23810,99.1200,98.560000


### Regressions

In [ ]:
# Separate the features (X) and the target variable (y)
X = prereg_df_imputed.iloc[:, 1:]
y = prereg_df_imputed.iloc[:, 0]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using sklearn for OLS regression
# Fit the model
reg_sklearn = LinearRegression()
reg_sklearn.fit(X_train, y_train)

# Generate predictions on the test set
y_pred_sklearn = reg_sklearn.predict(X_test)

# Assess the model
# Here we can print out the coefficients, intercept, and performance metrics as needed
# For example:
print("Coefficients:", reg_sklearn.coef_)
print("Intercept:", reg_sklearn.intercept_)
# Add any additional performance metrics you want to evaluate

# Using statsmodels for OLS regression to get a detailed summary
# Add a constant term to the predictor to get the intercept
X_train_sm = sm.add_constant(X_train)

# Fit the model
model_sm = sm.OLS(y_train, X_train_sm).fit()

# Print out the statistics
model_summary = model_sm.summary()
print(model_summary)


# Calculate and print performance metrics
mse = mean_squared_error(y_test, y_pred_sklearn)
r2 = r2_score(y_test, y_pred_sklearn)

print(f"Mean Squared Error (MSE): {mse}")
print(f"out-of-sample R-squared: {r2}")

Coefficients: [ 7.90011594e-03  1.13046754e-05  7.41484620e-04 -1.55612933e-06
  8.68460117e-10 -1.97984855e-08  4.17566389e-08 -3.04486101e-05
  2.27833203e-09  4.39770277e-09  6.38887496e-05  0.00000000e+00
 -6.45749195e-09 -8.94261027e-04 -2.26152929e-06 -3.62699950e-08
  1.97439817e-09 -4.39971434e-09 -2.55629001e-10 -9.34863162e-05
 -3.24365833e-08 -8.87944666e-08 -1.72373540e-08 -7.57533862e-08
 -5.50250439e-11 -1.14368955e-07 -1.53622462e-06]
Intercept: 475477.4950836906
                                OLS Regression Results                               
Dep. Variable:     Mangrove Change From 1996   R-squared:                       1.000
Model:                                   OLS   Adj. R-squared:                   -inf
Method:                        Least Squares   F-statistic:                       nan
Date:                       Wed, 21 Feb 2024   Prob (F-statistic):                nan
Time:                               07:54:54   Log-Likelihood:                 24.589
N

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


# **The following code uses PCA to rank feature importance of this dataset**





In [ ]:
X

,"Road Indicators Network, Total (km)","Rail Lines, Total Route (km)",International Tourist Arrivals ('000),International Tourism Receipts ($ million),GDP at current prices,Agriculture (% of GDP),Industry (% of GDP),Per capita GDP,Deforestation Rate (average % change),Agricultural Land (% of total land area),...,"Carbon Dioxide Emissions, Per Unit of Manufacturing Value-Added (kg of CO₂ equivalent per constant 2015 $)","Carbon Dioxide Emissions, Per Unit of GDP (PPP) (kg of CO₂ equivalent per constant 2017 $)",Number of Deaths Due to Disaster,"Proportion of Population Using Safely Managed Drinking Water, Rural (%)","Proportion of Population Using Safely Managed Sanitation, Total (%)","Proportion of Population Using Safely Managed Sanitation, Urban (%)","Proportion of Population Using Safely Managed Sanitation, Rural (%)","Level of Water Stress, Freshwater Withdrawal as a Proportion of Available Freshwater Resources (%)",Proportion of Population Covered by 2G Mobile Networks (%),Proportion of Population Covered by 3G Mobile Networks (%)
0,4.016401e+06,63273.000000,5082000.0,1.073000e+10,4.898662e+13,18.273885,33.714518,43046.23953,-0.280000,60.413226,...,1.510,0.292000,8340.0,39.0,19.0,27.0,16.0,65.34913,70.0000,0.000000
1,4.109592e+06,64015.000000,5283000.0,1.183200e+10,5.514152e+13,17.898843,33.191183,47782.94961,-0.280000,60.459305,...,1.505,0.302000,7341.0,40.0,21.0,28.0,19.0,65.73012,92.0225,0.000000
2,4.471510e+06,63974.000000,5168000.0,1.113600e+10,6.366407e+13,17.841424,33.160573,54413.73113,-0.280000,60.563906,...,1.564,0.308000,6148.0,42.0,23.0,28.0,21.0,66.11111,83.0000,0.080000
3,4.582439e+06,64460.000000,5776000.0,1.449000e+10,7.634472e+13,18.350431,33.114153,64371.60290,-0.270000,60.397418,...,1.628,0.301000,7489.0,43.0,25.0,29.0,24.0,66.49209,92.0225,59.162222
4,5.472144e+06,66252.000000,13284000.0,2.101300e+10,1.377187e+14,17.714686,29.951484,107341.18380,-0.380000,60.431725,...,1.487,0.284000,7313.0,51.0,36.0,33.0,37.0,66.49209,95.0000,74.000000
5,5.603293e+06,66918.000000,14570000.0,2.242700e+10,1.539167e+14,18.035273,29.337983,118488.59900,-0.380000,60.399100,...,1.435,0.267000,6942.0,52.0,38.0,34.0,40.0,66.49209,96.0000,79.670000
6,5.909561e+06,66935.000000,15543000.0,2.736500e+10,1.709004e+14,18.250271,29.207802,130061.20520,-0.370000,60.399100,...,1.444,0.265000,7317.0,54.0,40.0,35.0,42.0,66.49209,97.0000,88.000000
7,6.215797e+06,67415.000000,17423000.0,2.856800e+10,1.889967e+14,17.641353,29.064108,142424.02750,-0.370000,60.399100,...,1.349,0.262000,6859.0,56.0,42.0,36.0,45.0,66.49209,97.0000,94.000000
8,6.386297e+06,67956.000000,17914000.0,3.072000e+10,2.007486e+14,18.296615,26.900975,149700.63980,-0.370000,60.399100,...,1.416,0.252000,7505.0,57.0,44.0,37.0,48.0,66.49209,99.0600,98.150000
9,5.196337e+06,65688.666667,6330000.0,1.303600e+10,1.980091e+14,19.988549,26.899028,146087.24710,-0.331111,60.429109,...,1.482,0.281444,6347.0,56.0,46.0,37.0,51.0,66.23810,99.1200,98.560000


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)

# Apply PCA
pca = PCA(n_components=10)
pca.fit(X_std)
components = pca.components_
print(pca.components_)

# Variance explained by each principal component
explained_variance = pca.explained_variance_ratio_

# Print the explained variance
print(pca.explained_variance_ratio_)

[[-2.26198057e-01 -2.27480251e-01 -2.04728845e-01 -2.03268262e-01
  -2.26957971e-01 -3.23688666e-02  2.15831802e-01 -2.28233672e-01
   2.13507698e-01  1.25466133e-01 -1.46114526e-01 -0.00000000e+00
  -6.87558541e-02 -2.28096377e-01 -2.19000896e-01 -1.74873711e-01
   2.02427348e-01  1.75350279e-01  2.10216738e-01  5.73558849e-02
  -2.29509024e-01 -2.25697523e-01 -2.26342230e-01 -2.24256177e-01
  -1.85096231e-01 -1.90871875e-01 -2.22746399e-01]
 [-9.13345042e-02 -9.70327501e-02 -2.52126918e-01 -2.51934656e-01
   8.82909146e-02  4.22001383e-01 -1.34048589e-01  8.12176252e-02
   1.05609247e-01  2.25222480e-01 -2.31139988e-01 -0.00000000e+00
   4.48864004e-01  9.92458511e-02 -6.16965721e-03  7.87770858e-02
   1.05940517e-01  1.65730379e-01  1.73285046e-01 -4.12712553e-01
   7.73783162e-02  1.24166149e-01  9.77724784e-02  1.34303800e-01
   4.59345574e-02  1.32070557e-01  6.54032921e-02]
 [-5.66368410e-02 -5.13525923e-02 -8.87874641e-02 -6.42101754e-02
   1.01146890e-01  4.42705738e-01 -1.511

In [ ]:
#print out the ranked importance of each feature within each of the 10 priciple components
feature_rank = []
for i in range(10):
  print("For the ", i, "th component: ")
  features = list(range(0,27))
  curr_comp = pca.components_[i].copy()
  features.sort(key=lambda x: abs(curr_comp[x]))
  feature_rank.append(features)
  print(features)

For the  0 th component: 
[11, 5, 19, 12, 9, 10, 15, 17, 24, 25, 16, 3, 2, 18, 8, 6, 14, 26, 23, 21, 0, 22, 4, 1, 13, 7, 20]
For the  1 th component: 
[11, 14, 24, 26, 20, 15, 7, 4, 0, 1, 22, 13, 8, 16, 21, 25, 6, 23, 17, 18, 9, 10, 3, 2, 19, 5, 12]
For the  2 th component: 
[11, 20, 1, 14, 13, 0, 16, 3, 8, 12, 21, 26, 17, 2, 7, 23, 4, 22, 25, 6, 18, 10, 24, 19, 9, 15, 5]
For the  3 th component: 
[11, 15, 13, 21, 23, 3, 1, 7, 20, 22, 8, 4, 0, 2, 5, 25, 18, 6, 16, 26, 19, 14, 12, 24, 17, 10, 9]
For the  4 th component: 
[11, 21, 4, 7, 2, 20, 9, 23, 18, 22, 5, 6, 0, 13, 1, 19, 26, 3, 15, 8, 14, 25, 24, 10, 17, 16, 12]
For the  5 th component: 
[11, 4, 18, 3, 12, 23, 7, 1, 14, 13, 5, 21, 6, 22, 20, 2, 26, 15, 0, 19, 9, 24, 8, 10, 16, 17, 25]
For the  6 th component: 
[11, 16, 22, 2, 1, 17, 20, 23, 26, 21, 7, 4, 5, 13, 12, 9, 24, 18, 0, 6, 3, 10, 14, 25, 15, 19, 8]
For the  7 th component: 
[11, 21, 7, 4, 20, 12, 1, 22, 25, 13, 23, 8, 0, 2, 15, 6, 24, 16, 18, 26, 19, 14, 3, 17, 9, 5, 10]


In [ ]:
#Now: USE RANK OF IMPORTANCE OF EACH FEATURE IN EACH COMPONENT AND VAR CAPTURED BY COMPONENT
#TO WEIGHT FEATURE IMPORTANCE
feature_keys = list(range(0,27))
feature_rank_dict = {k:v for k,v in zip(feature_keys, [0 for i in range(27)])}
for component in range(len(feature_rank)):
  var_weight = pca.explained_variance_ratio_[component]
  rank_weight = 27
  for f in feature_rank[component]:
    feature_rank_dict[f] += var_weight * (rank_weight)
    rank_weight -= 1
feature_weights = list(feature_rank_dict.values())
feature_weights.sort(reverse=True)
ranked_features = [list(feature_rank_dict.values()).index(i) for i in feature_weights]

In [ ]:
#features in order of importance to capturing variance within data
print([X.columns[i] for i in ranked_features])

['Average Proportion of Marine Key Biodiversity Areas Covered by Protected Areas (%)', 'Agriculture (% of GDP)', 'Proportion of Medium and High-Tech Industry Value Added in Total Value Added (%)', 'Manufacturing Value-Added, As a Proportion of GDP (%)', 'Number of Deaths Due to Disaster', 'Agricultural Land (% of total land area)', 'Level of Water Stress, Freshwater Withdrawal as a Proportion of Available Freshwater Resources (%)', 'Amount of Water- and Sanitation-Related Official Development Assistance as Part of a Government-Coordinated Spending Plan ($ million)', 'Carbon Dioxide Emissions, Per Unit of Manufacturing Value-Added (kg of CO₂ equivalent per constant 2015 $)', 'Research and Development Expenditure as a Proportion of GDP (%)', 'Proportion of Population Covered by 2G Mobile Networks (%)', 'International Tourism Receipts ($ million)', "International Tourist Arrivals ('000)", 'Deforestation Rate (average % change)', 'Total Official International Support to Infrastructure (con

### Feature Engineering

From the regression output above, we noticed a very high R-squared and a significantly lower out-of-sample R-squared as before, which indicates severe overfitting. Therefore, we need to refine the data and remove some features. For this project, we are adopting the following **iterative** approach:

1. (non-iterative) Check the VIF in the data we ran regression on above;
2. Find the features with high VIF values;
3. Compare the correlations that the two variables have with the y variable (by comparing the regression results);
4. Remove the feature that seems to be worse;
5. Test for VIF again and see if there's any improvement compared to the previous (or initial) VIF values.
6. Iterate process for step 2 - 6, until all VIF values are below 5 (which is a relatively reasonable threshold)

In [ ]:
# Add a constant term to the predictor to get the intercept for VIF calculation
X_train_with_const = sm.add_constant(X_train)

# Calculate VIFs for each variable
vif_data = pd.DataFrame()
vif_data["feature"] = X_train_with_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_train_with_const.values, i) for i in range(X_train_with_const.shape[1])]

# Display the VIFs
print(vif_data)

                                              feature           VIF
0                 Road Indicators Network, Total (km)  3.382986e+11
1                        Rail Lines, Total Route (km)  5.925789e+12
2               International Tourist Arrivals ('000)  5.298353e+13
3          International Tourism Receipts ($ million)  1.801440e+15
4                               GDP at current prices           inf
5                              Agriculture (% of GDP)  2.238370e+12
6                                 Industry (% of GDP)  1.218836e+13
7                                      Per capita GDP           inf
8               Deforestation Rate (average % change)  3.182756e+13
9            Agricultural Land (% of total land area)  4.587278e+09
10  Amount of Water- and Sanitation-Related Offici...  2.006058e+13
11  Average Proportion of Marine Key Biodiversity ...  0.000000e+00
12  Manufacturing Value-Added, As a Proportion of ...  3.816610e+13
13  Manufacturing Value-Added, Per Capita (at co

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


In [ ]:
# Extract the dependent variable and the two independent variables into separate series
y_variable = prereg_df_imputed['Mangrove Change From 1996']  # Dependent variable
first_variable = prereg_df_imputed['GDP at current prices']  # First independent variable
second_variable = prereg_df_imputed['Per capita GDP']  # Second independent variable

# Compute Pearson correlation coefficients
correlation_first = first_variable.corr(y_variable)
correlation_second = second_variable.corr(y_variable)

print(f"Correlation of the first variable with the dependent variable: {correlation_first}")
print(f"Correlation of the second variable with the dependent variable: {correlation_second}")

# Run individual OLS regressions for each variable
model_first = sm.OLS(y_variable, sm.add_constant(first_variable)).fit()
model_second = sm.OLS(y_variable, sm.add_constant(second_variable)).fit()

print(model_first.summary())
print(model_second.summary())

- Correlation Coefficient: Measures the strength and direction of the relationship between an independent variable and the dependent variable. We prefer a higher absolute value, indicating a stronger relationship.

- R-squared (R²): Indicates the proportion of the variance in the dependent variable that is predictable from the independent variable(s). We prefer a higher R², which suggests a greater explanatory power of the model.

- Adjusted R-squared: Similar to R² but adjusts for the number of predictors in the model. A higher adjusted R² is preferred, especially when comparing models with a different number of independent variables.

- F-statistic: Evaluates the overall significance of the model. A higher F-statistic is preferred as it suggests the model has greater explanatory power.

- Prob (F-statistic): The p-value for the F-statistic. We prefer a smaller value, indicating that the model is statistically significant.

- Coefficients: Represent the change in the dependent variable for a one-unit change in the independent variable(s). Larger absolute values are generally preferred, but the sign should be consistent with the expected relationship.

- P-value of Coefficients (P>|t|): Assesses the significance of individual regression coefficients. Smaller values are preferred as they suggest the variable is a significant predictor of the dependent variable.

- Confidence Interval: Provides a range within which the true regression coefficient is likely to fall. We prefer narrower intervals, indicating more precision in the coefficient estimates.

- Condition Number: Used to diagnose multicollinearity. Lower values are preferred as they suggest less multicollinearity.

- Durbin-Watson Statistic: Tests for autocorrelation in the residuals. We prefer values close to 2, which indicate no autocorrelation.

- Omnibus/Prob(Omnibus): Tests the combined skewness and kurtosis of the residuals. We prefer a higher p-value (p > 0.05) suggesting that the residuals are normally distributed.

- Jarque-Bera (JB)/Prob(JB): Tests the normality of the residuals. We prefer a higher p-value indicating that the residuals are normally distributed, which is a desirable property in regression analysis.


In [ ]:
# Make a copy of the prereg_df_imputed DataFrame to test the column removing process
reg_df_imputed = prereg_df_imputed.copy()

# Remove the column 'GDP at current prices'
reg_df_imputed.drop('GDP at current prices', axis=1, inplace=True)
print(reg_df_imputed.shape)
reg_df_imputed.head()

In [ ]:
# Calculate the correlation matrix
corr = reg_df_imputed.corr()

# Create a mask to display only the lower triangle of the matrix
mask = np.triu(np.ones_like(corr, dtype=bool))

# Create a pairplot
pairplot_fig = sns.pairplot(reg_df_imputed)

# Loop through axes to plot the upper triangle with the correlation values
for i, j in zip(*np.triu_indices_from(pairplot_fig.axes, 1)):
    pairplot_fig.axes[i, j].set_visible(False)
    txt = f'{corr.iloc[i, j]:.2f}'
    pairplot_fig.axes[i, j].text(0.5, 0.5, txt, transform=pairplot_fig.axes[i, j].transAxes,
                                  ha='center', va='center', color='red', fontsize='large')

plt.show()

# Define a threshold for high correlation
threshold = 0.75  # For example, we choose 0.75 as the high correlation threshold

# Find pairs with high correlation
high_corr_pairs = [(reg_df_imputed.columns[x], reg_df_imputed.columns[y], corr.iloc[x, y])
                   for x in range(len(corr.columns))
                   for y in range(x+1, len(corr.columns))
                   if abs(corr.iloc[x, y]) > threshold]

# Print or store the high correlation pairs
for pair in high_corr_pairs:
    print(f"{pair[0]} and {pair[1]} have a correlation of {pair[2]:.2f}")

### Collecting data on more variables